In [2]:
import glob
from symspellpy import SymSpell, Verbosity
from symspellpy.editdistance import EditDistance, DistanceAlgorithm
import pkg_resources
from itertools import islice
import re
from unidecode import unidecode
import pandas as pd
from collections import Counter
from datetime import datetime
import numpy as np
import json

In [11]:
df_licitacao = pd.read_csv("../data/items_druid_original.csv.zip", sep=";")
df_nf = pd.read_csv("../data/output/nota_fiscal_fasttext/f03_items.csv.zip", sep=";")

In [57]:
display(df_licitacao)
display(df_nf)

df = df_nf

,palavras,unidades_medida,numeros,cores,materiais,tamanho,quantidade,preco,dsc_unidade_medida,original,original_prep,ano,licitacao,licitacao_item,item_id
0,"['encunhamento', 'aperto', 'alvenaria', 'tijol...","['x', 'cm', 'c']","['2.00e+00', '4.00e+00', '1.00e+01', '2.00e+01...",[],[],[],[],15.87,METRO,5.2.4 ENCUNHAMENTO (APERTO DE ALVENARIA) EM TI...,"['2', '4', 'encunhamento', 'aperto', 'alvenari...",2017,297095,10000108,0
1,"['piso', 'cimentado', 'desempenado', 'com', 'a...","['cm', 'm']","['1.00e+00', '3.00e+00', '0.00e+00', '2.00e+00...",[],[],[],[],42.78,M²,10.1.3 PISO CIMENTADO DESEMPENADO COM ACABAMEN...,"['1', '3', 'piso', 'cimentado', 'desempenado',...",2017,297095,10000172,1
2,"['chuveiro', 'maxi', 'ducha', 'lorenzetti', 'c...",[],"['8.01e+03', '1.51e+01']",[],[],[],[],58.78,UNIDADE,"15.12 CHUVEIRO MAXI DUCHA, LORENZETTI,COM MANG...","['chuveiro', 'maxi', 'ducha', 'lorenzetti', 'c...",2017,297095,10000304,2
3,"['tubo', 'soldavel', 'inclusive', 'conexoes']",['mm'],"['2.50e+01', '1.91e+01']",[],['pvc'],[],[],3.42,METRO,"19.1 TUBO PVC SOLDAVEL 25MM, INCLUSIVE CONEXOES","['tubo', 'pvc', 'soldavel', '25', 'mm', 'inclu...",2017,297095,10000454,3
4,"['tomada', 'modular', 'rj', 'categoria', 'comp...",[],"['4.00e+00', '1.00e+00', '4.50e+01', '6.00e+00...",[],[],[],[],12.37,UNIDADE,20.4.1 TOMADA MODULAR RJ-45 CATEGORIA 6 (COMPL...,"['4', '1', 'tomada', 'modular', 'rj', '45', 'c...",2017,297095,10000472,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10902491,"['essencia', 'aromatizante', 'vd', 'erva', 'do...",['ml'],['1.00e+02'],[],['eucalipto'],[],[],3.00,VIDRO,ESSENCIA AROMATIZANTE VD /100ML ( ERVA DOCE E ...,"['essencia', 'aromatizante', 'vd', '100', 'ml'...",2017,297050,9998875,10902491
10902492,"['papel', 'camurca', 'variadas']","['x', 'cm']","['4.00e+01', '6.00e+01']",[],[],[],[],0.70,FOLHA,PAPEL CAMURCA 40X60 CM CORES VARIADAS,"['papel', 'camurca', '40', 'x', '60', 'cm', 'c...",2017,297052,9998909,10902492
10902493,"['torta', 'tubos', 'leque']",[],"['5.00e+01', '1.50e+00']",[],[],[],[],257.14,UNIDADE,"TORTA 50 TUBOS 1,5 EM LEQUE","['torta', '50', 'tubos', '1.5', 'leque']",2017,297059,9999325,10902493
10902494,"['chave', 'liga', 'desliga', 'refletor', 'pedal']",[],[],[],[],[],[],47.00,UNID,CHAVE DE LIGA DESLIGA REFLETOR PEDAL,"['chave', 'liga', 'desliga', 'refletor', 'pedal']",2017,297085,9999725,10902494


,palavras,unidades_medida,numeros,cores,materiais,tamanho,quantidade,preco,dsc_unidade_medida,original,original_prep,ano,licitacao,licitacao_item,item_id
0,"['prestacao', 'servico', 'eletrico', 'veiculo'...",[],['2.41e+03'],[],[],[],[],63.00,hora,PRESTACAO DE SERVICOS ELETRICOS EM VEICULOS LE...,"['prestacao', 'servico', 'eletrico', 'veiculo'...",2014,91611180,38706974,0
1,"['oleo', 'diesel', 'combustivel']",[],"['1.00e+01', '2.57e+03']",[],[],[],[],2.71,litro,OLEO DIESEL COMBUSTIVEL S10 2567,"['oleo', 'diesel', 'combustivel', '10', '2567']",2014,91611231,38706975,1
2,"['graxo', 'brif', 'ngl2']",[],"['9.00e+00', '2.80e+03']",[],[],[],[],30.00,unidade,G 052737Q9 GRAXA LUBRIF NGL2 2797,"['g', 'q', '9', 'graxo', 'brif', 'ngl2', '2797']",2014,91638555,38706976,2
3,"['oleo', 'dif']",[],"['9.00e+00', '8.50e+01', '1.40e+02', '2.80e+03']",[],[],[],[],25.00,unidade,G 052815Q9 OLEO DIF 85W140 2798,"['g', 'q', '9', 'oleo', 'dif', '85', 'w', '140...",2014,91638555,38706977,3
4,['pneu'],[],"['1.30e+01', '4.72e+02']",[],[],[],[],180.00,unidade,PNEU 16570R13 472,"['pneu', 'r', '13', '472']",2014,91611257,38706978,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3679112,"['servico', 'mecanico', 'motor', 'freio']",[],['7.00e+03'],[],[],[],[],75.00,hora,SERVICOS MECANICOS DE MOTOR E FREIOS 7001,"['servico', 'mecanico', 'motor', 'freio', '7001']",2020,121762479,44934348,3679112
3679113,"['contratacao', 'software', 'adiministracao', ...",[],['2.45e+03'],[],[],[],[],6300.00,mes,CONTRATACAO DE SOFTWARE DE ADIMINISTRACAO PUBL...,"['contratacao', 'software', 'adiministracao', ...",2020,121762480,44934349,3679113
3679114,"['escada', 'extensivo', 'degrau', 'rebitar']",[],"['3.60e+02', '6.00e+02', '8.74e+03']",[],[],[],[],588.00,unidade,ESCADA EXTENSIVA 360 X 600 DEGRAU REBITADO 8740,"['escada', 'extensivo', '360', 'x', '600', 'de...",2020,121762481,44934350,3679114
3679115,"['oleo', 'diesel', 'combustivel', 'comum']",[],['4.92e+02'],[],[],[],[],3.75,litro,OLEO DIESEL COMBUSTIVEL COMUM 492,"['oleo', 'diesel', 'combustivel', 'comum', '492']",2020,121762482,44934351,3679115


In [46]:
has_number = re.compile("[0-9]")

def normalize(text):
    text = text.lower()
    text = unidecode(text)
    text = re.sub("\d+\.\d+(?:.\d+)+", " ", text) # remove numeração de itens em lista, eg, "5.2.4" ENCUNHAME...
    text = re.sub("[^a-z0-9]", " ", text)
    tokens = text.strip().split()
    tokens = [t for t in tokens if not has_number.search(t) and len(t) >= 3]
    return tokens

# Build the dictionary

In [58]:
all_tokens = []
for i, row in df.iterrows():
    all_tokens.extend(normalize(row.original))

In [59]:
freq = Counter(all_tokens)

In [60]:
for key, v in freq.most_common():
    if key.endswith("/"):
        print(key, v)

In [61]:
with open("../data/dicionario/br-words.txt") as f:
    lines = f.readlines()

with open("../data/dicionario/br-words-preprocessed.txt", "wt", encoding="utf-8") as f:
    lines = set([unidecode(l.lower().strip()) for l in lines]) & set(freq.keys())
    diff = set(freq.keys()) - set([unidecode(l.lower().strip()) for l in lines])
    for l in sorted(lines):
        f.write(f"{l} {freq[l]}\n")
            

# Select valid replacements

In [62]:
sym_spell = SymSpell()
sym_spell.load_dictionary("../data/dicionario/br-words-preprocessed.txt", 0, 1)
replacements = {}
for word in diff:
    suggestions = list(sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=1, ignore_token=r"\w+\d"))
    if len(suggestions) == 1:
        suggestion = suggestions[0]
        term = suggestion._term
        count = suggestion._count
        if count > 1000:
            print(f"replace \"{word}\" by \"{term}\"")
            replacements[word] = term
        


replace "rioquimica" by "bioquimica"
replace "diateiro" by "dianteiro"
replace "adapitador" by "adaptador"
replace "amortecedro" by "amortecedor"
replace "puntura" by "pintura"
replace "medelo" by "modelo"
replace "lucares" by "lugares"
replace "cruxeta" by "cruzeta"
replace "pivu" by "pivo"
replace "uric" by "urico"
replace "embregem" by "embreagem"
replace "cvhaves" by "chaves"
replace "eatraves" by "atraves"
replace "fontd" by "fonte"
replace "alvanizado" by "galvanizado"
replace "aluminioe" by "aluminio"
replace "deametro" by "diametro"
replace "gerenciamneto" by "gerenciamento"
replace "interrupto" by "interruptor"
replace "aquiisicao" by "aquisicao"
replace "revistida" by "revestida"
replace "ccaixa" by "caixa"
replace "execesso" by "excesso"
replace "conservcao" by "conservacao"
replace "amistura" by "mistura"
replace "prancehta" by "prancheta"
replace "eambientes" by "ambientes"
replace "ontratacao" by "contratacao"
replace "extratp" by "extrato"
replace "oralb" by "oral"
repla

replace "grandese" by "grandes"
replace "apraoximada" by "aproximada"
replace "limpezas" by "limpeza"
replace "plantica" by "plastica"
replace "aquipamentos" by "equipamentos"
replace "atampa" by "tampa"
replace "dentesw" by "dentes"
replace "lsolidos" by "solidos"
replace "lauminio" by "aluminio"
replace "ligt" by "liga"
replace "residios" by "residuos"
replace "cpmprimido" by "comprimido"
replace "ecoloracao" by "coloracao"
replace "esterografica" by "esferografica"
replace "stampo" by "tampo"
replace "esritorio" by "escritorio"
replace "pinaca" by "pinca"
replace "crecarga" by "recarga"
replace "gaxetas" by "gavetas"
replace "resistene" by "resistente"
replace "fechamewnto" by "fechamento"
replace "vimeo" by "video"
replace "bromato" by "brometo"
replace "joeho" by "joelho"
replace "sabenete" by "sabonete"
replace "vauvula" by "valvula"
replace "resistinte" by "resistente"
replace "posh" by "pos"
replace "sobor" by "sabor"
replace "proteetor" by "protetor"
replace "prfeitura" by "pr

replace "adsivo" by "adesivo"
replace "eatualmente" by "atualmente"
replace "carboidrat" by "carboidrato"
replace "procediimento" by "procedimento"
replace "aramario" by "armario"
replace "uruana" by "urbana"
replace "tamamnho" by "tamanho"
replace "fehcada" by "fechada"
replace "estaril" by "esteril"
replace "altrnador" by "alternador"
replace "quache" by "guache"
replace "llitros" by "litros"
replace "grasco" by "frasco"
replace "necesario" by "necessario"
replace "frequancia" by "frequencia"
replace "qualidadae" by "qualidade"
replace "farinh" by "farinha"
replace "acondicioanado" by "acondicionado"
replace "pastilh" by "pastilha"
replace "lampad" by "lampada"
replace "powder" by "poder"
replace "acriancas" by "criancas"
replace "naturtal" by "natural"
replace "balaceamento" by "balanceamento"
replace "aluminion" by "aluminio"
replace "conformidad" by "conformidade"
replace "ultilizacao" by "utilizacao"
replace "bisaga" by "bisnaga"
replace "erevestimento" by "revestimento"
replace 

replace "unidadesn" by "unidades"
replace "flexcivel" by "flexivel"
replace "calduladora" by "calculadora"
replace "acrilio" by "acrilico"
replace "protetota" by "protetora"
replace "fanti" by "anti"
replace "gavanizada" by "galvanizada"
replace "catergoria" by "categoria"
replace "ampolasl" by "ampolas"
replace "papael" by "papel"
replace "secagema" by "secagem"
replace "nascional" by "nacional"
replace "ptoteinas" by "proteinas"
replace "utillizada" by "utilizada"
replace "aderent" by "aderente"
replace "einformacoes" by "informacoes"
replace "procedencias" by "procedencia"
replace "admtindo" by "admitindo"
replace "eleboracao" by "elaboracao"
replace "domicialiar" by "domiciliar"
replace "lubrifucante" by "lubrificante"
replace "fabricacaoe" by "fabricacao"
replace "financceiro" by "financeiro"
replace "supeficie" by "superficie"
replace "laltura" by "altura"
replace "valvuva" by "valvula"
replace "celeular" by "celular"
replace "alimentecao" by "alimentacao"
replace "cumidade" by "

replace "esquero" by "esquerdo"
replace "algoao" by "algodao"
replace "tescido" by "tecido"
replace "resrvatorio" by "reservatorio"
replace "tecnilogico" by "tecnologico"
replace "vegtais" by "vegetais"
replace "refigerante" by "refrigerante"
replace "pomad" by "pomada"
replace "acp" by "aco"
replace "cirurugico" by "cirurgico"
replace "estrrutura" by "estrutura"
replace "acces" by "acoes"
replace "entrre" by "entre"
replace "hip" by "hiv"
replace "acriico" by "acrilico"
replace "fudamental" by "fundamental"
replace "pojeto" by "projeto"
replace "adptador" by "adaptador"
replace "event" by "evento"
replace "isolameno" by "isolamento"
replace "lixoz" by "lixo"
replace "madras" by "maduras"
replace "docx" by "doce"
replace "resisitente" by "resistente"
replace "passgeiros" by "passageiros"
replace "quallidade" by "qualidade"
replace "tkl" by "tal"
replace "gorsso" by "grosso"
replace "organio" by "organico"
replace "adapatdor" by "adaptador"
replace "feminico" by "feminino"
replace "reto

...

replace "recupercao" by "recuperacao"
replace "gasolin" by "gasolina"
replace "parafudo" by "parafuso"
replace "passseio" by "passeio"
replace "simv" by "sim"
replace "colordio" by "colorido"
replace "aliementos" by "alimentos"
replace "comoposicao" by "composicao"
replace "lubrrificante" by "lubrificante"
replace "eletrosatica" by "eletrostatica"
replace "transmssao" by "transmissao"
replace "desinfeacao" by "desinfeccao"
replace "tranmissao" by "transmissao"
replace "abracadeiira" by "abracadeira"
replace "naional" by "nacional"
replace "especifiacao" by "especificacao"
replace "annel" by "anel"
replace "timento" by "cimento"
replace "unidadeso" by "unidades"
replace "bistur" by "bisturi"
replace "agricolla" by "agricola"
replace "montsagem" by "montagem"
replace "qadesivo" by "adesivo"
replace "progeto" by "projeto"
replace "circulaca" by "circulacao"
replace "xaropo" by "xarope"
replace "esverdiada" by "esverdeada"
replace "passeiro" by "passeio"
replace "mamgueira" by "mangueira"


replace "tvl" by "tal"
replace "retrona" by "retorna"
replace "piimenta" by "pimenta"
replace "nasp" by "nas"
replace "profunidade" by "profundidade"
replace "marcaneta" by "macaneta"
replace "ecobertura" by "cobertura"
replace "lurificante" by "lubrificante"
replace "acbamento" by "acabamento"
replace "teciso" by "tecido"
replace "volant" by "volante"
replace "eletrivcos" by "eletricos"
replace "lvaso" by "vaso"
replace "amareta" by "amarela"
replace "sisntetica" by "sintetica"
replace "aguosa" by "aquosa"
replace "traseiira" by "traseira"
replace "ermentado" by "fermentado"
replace "panpel" by "papel"
replace "cromados" by "cromado"
replace "frabricacao" by "fabricacao"
replace "indl" by "indo"
replace "pretro" by "preto"
replace "cobertuta" by "cobertura"
replace "conformdiade" by "conformidade"
replace "estadu" by "estado"
replace "conseva" by "conserva"
replace "ventiladdor" by "ventilador"
replace "touck" by "touca"
replace "compoato" by "composto"
replace "artesanto" by "artesan

replace "coxiim" by "coxim"
replace "four" by "for"
replace "eapoio" by "apoio"
replace "emtc" by "etc"
replace "denti" by "dente"
replace "enchiment" by "enchimento"
replace "visulizacao" by "visualizacao"
replace "silindro" by "cilindro"
replace "impressoara" by "impressora"
replace "inclolor" by "incolor"
replace "eespecificacoes" by "especificacoes"
replace "atomic" by "atomico"
replace "ppublica" by "publica"
replace "reraros" by "reparos"
replace "termperatura" by "temperatura"
replace "repollho" by "repolho"
replace "procekdimento" by "procedimento"
replace "grandis" by "grandes"
replace "informaica" by "informatica"
replace "rerigerante" by "refrigerante"
replace "sinjetor" by "injetor"
replace "receheio" by "recheio"
replace "horaro" by "horario"
replace "ceebola" by "cebola"
replace "integrale" by "integral"
replace "densdade" by "densidade"
replace "crurgica" by "cirurgica"
replace "aprezentacao" by "apresentacao"
replace "sejoa" by "seja"
replace "pimpressora" by "impressor

replace "orpos" by "corpos"
replace "fostato" by "fosfato"
replace "deman" by "demao"
replace "gpressao" by "pressao"
replace "periodi" by "periodo"
replace "paraistas" by "parasitas"
replace "diverssas" by "diversas"
replace "vejetal" by "vegetal"
replace "succ" by "suco"
replace "pincotado" by "picotado"
replace "ondontologica" by "odontologica"
replace "indcador" by "indicador"
replace "assisntencia" by "assistencia"
replace "indivudualmente" by "individualmente"
replace "galvabizado" by "galvanizado"
replace "novaes" by "novas"
replace "tcl" by "tal"
replace "aproximadaente" by "aproximadamente"
replace "rvegetais" by "vegetais"
replace "liquiso" by "liquido"
replace "coximl" by "coxim"
replace "coloaracao" by "coloracao"
replace "mgrade" by "grade"
replace "intermet" by "internet"
replace "niversal" by "universal"
replace "alminio" by "aluminio"
replace "servidoras" by "servidores"
replace "confeccinado" by "confeccionado"
replace "cepai" by "cepa"
replace "femenino" by "feminino"

replace "carderno" by "caderno"
replace "lamapda" by "lampada"
replace "caractristicos" by "caracteristicos"
replace "moptor" by "motor"
replace "peasada" by "pesada"
replace "pincell" by "pincel"
replace "odontalogica" by "odontologica"
replace "encostto" by "encosto"
replace "sebastia" by "sebastiao"
replace "elarvas" by "larvas"
replace "lomga" by "longa"
replace "enm" by "nem"
replace "vermlho" by "vermelho"
replace "cartuco" by "cartucho"
replace "plastifiado" by "plastificado"
replace "serrvicos" by "servicos"
replace "mbranco" by "branco"
replace "mesida" by "medida"
replace "esterrilizacao" by "esterilizacao"
replace "antigebo" by "antigeno"
replace "frentex" by "frente"
replace "olume" by "volume"
replace "acodicionamento" by "acondicionamento"
replace "adicaoc" by "adicao"
replace "acaracteristicas" by "caracteristicas"
replace "eucaalipto" by "eucalipto"
replace "vermerlha" by "vermelha"
replace "parelelo" by "paralelo"
replace "ambilancia" by "ambulancia"
replace "polimer" 

In [63]:
total = 0
for word in replacements:
    total += freq[word]
    
100 * total / sum(freq.values())

0.8423059378531701

In [64]:
with open("../data/dicionario/replacement_nf.json", "wt", encoding="utf-8") as f:
    json.dump(replacements, f)

In [56]:
# Erros na base de licitações: 0.9832811647717699%